In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, LSTM
from keras.utils import np_utils
#from keras.utils.visualize_util import plot
from keras.callbacks import TensorBoard
import os

In [3]:
digit = "0123456789"
alpha = "abcdefghij"

In [4]:
char_set = list(set(digit + alpha))  # id -> char
char_dic = {w: i for i, w in enumerate(char_set)}

In [7]:
data_dim = len(char_set)  # one hot encoding size
seq_length = time_steps = 7
nb_classes = len(char_set)

In [11]:
# Build training date set
dataX = []
dataY = []

for i in range(1000):
    rand_pick = np.random.choice(10, 7)
    x = [char_dic[digit[c]] for c in rand_pick]
    y = [char_dic[alpha[c]] for c in rand_pick]
    dataX.append(x)
    dataY.append(y)

In [13]:
# One-hot encoding
dataX = np_utils.to_categorical(dataX, nb_classes=nb_classes)
# reshape X to be [samples, time steps, features]
dataX = np.reshape(dataX, (-1, seq_length, data_dim))

In [14]:
# One-hot encoding
dataY = np_utils.to_categorical(dataY, nb_classes=nb_classes)
# time steps
dataY = np.reshape(dataY, (-1, seq_length, data_dim))

In [16]:
dataX.shape , dataY.shape

((1000, 7, 20), (1000, 7, 20))

In [17]:
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE
# note: in a situation where your input sequences have a variable length,
# use input_shape=(None, nb_feature).
model.add(LSTM(32, input_shape=(time_steps, data_dim), return_sequences=False))

# For the decoder's input, we repeat the encoded input for each time step
model.add(RepeatVector(time_steps))
# The decoder RNN could be multiple layers stacked or a single layer

model.add(LSTM(32, return_sequences=True))

# For each of step of the output sequence, decide which character should
# be chosen
model.add(TimeDistributed(Dense(data_dim)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 32)            6784        lstm_input_1[0][0]               
____________________________________________________________________________________________________
repeatvector_1 (RepeatVector)    (None, 7, 32)         0           lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 7, 32)         8320        repeatvector_1[0][0]             
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 7, 20)         660         lstm_2[0][0]                     
___________________________________________________________________________________________

In [18]:
model.fit(dataX, dataY, nb_epoch=1000)

Epoch 1/1000
1000/1000 [==============================] - 2s - loss: 2.8988 - acc: 0.1470      
Epoch 2/1000
1000/1000 [==============================] - 0s - loss: 2.4915 - acc: 0.1826     